In [ ]:
import pandas as pd
import pysentiment2 as ps  # <--- Nouvelle librairie
from tqdm import tqdm

# 1. Configuration
FILE_PATH = '../data/headlines.csv'
START_DATE = '2005-01-01'  
END_DATE = '2008-12-31'

# 2. Chargement & Nettoyage (Ton code existant)
df_large = pd.read_csv(FILE_PATH)

col_text = [c for c in df_large.columns if 'title' in c.lower() or 'headline' in c.lower()][0]
col_date = [c for c in df_large.columns if 'date' in c.lower() or 'published' in c.lower()][0]

# Conversion date
df_large[col_date] = pd.to_datetime(df_large[col_date].astype(str), format='%Y%m%d', errors='coerce')
df_large = df_large.dropna(subset=[col_date])

# Filtre temporel
mask = (df_large[col_date] >= START_DATE) & (df_large[col_date] <= END_DATE)
df_filtered = df_large.loc[mask].copy()

print(f"Lignes trouvées : {len(df_filtered)}")

# 3. Initialisation du moteur Loughran-McDonald
lm = ps.LM()  # Charge le dictionnaire Finance

def get_lm_score(text):
    # LM a besoin de tokens (mots séparés), pas de phrases brutes
    tokens = lm.tokenize(str(text))
    score = lm.get_score(tokens)
    # Le score retourné est un dictionnaire : {'Positive': x, 'Negative': y, 'Polarity': z, ...}
    # Polarity = (Pos - Neg) / (Pos + Neg). 
    return score['Polarity']

# 4. Calcul (Si données présentes)
if len(df_filtered) > 0:
    tqdm.pandas()
    print("Calcul du sentiment Loughran-McDonald (Finance Specific)...")
    
    # Attention : C'est un peu plus lent que VADER car la tokenisation est plus stricte
    df_filtered['sentiment_score'] = df_filtered[col_text].progress_apply(get_lm_score)
    
    # 5. Agrégation
    df_filtered['date_only'] = df_filtered[col_date].dt.date
    daily_agg = df_filtered.groupby('date_only')['sentiment_score'].agg(['mean', 'count']).reset_index()
    
    # 6. Sauvegarde
    daily_agg.to_csv('../data/daily_sentiment_lm_final.csv', index=False)
    print("Fichier daily_sentiment_lm_final.csv généré !")
    
else:
    print("Aucune donnée trouvée après filtrage.")

Lignes trouvées : 370399
Calcul du sentiment Loughran-McDonald (Finance Specific)...


100%|██████████| 370399/370399 [00:18<00:00, 19546.65it/s]


Fichier daily_sentiment_lm_final.csv généré !


In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests, adfuller

# --- 1. PRÉPARATION (Ton code) ---
df_sent = pd.read_csv('../data/daily_sentiment_lm_final.csv')
df_fine = pd.read_csv('../data/financial_data_XLF.csv')

# Conversion dates
df_sent['date_only'] = pd.to_datetime(df_sent['date_only'])
df_fine['Date'] = pd.to_datetime(df_fine['Date']).dt.tz_localize(None)

# Fusion (Inner Join)
df_combined = pd.merge(df_fine, df_sent, left_on='Date', right_on='date_only', how='inner')

# --- 2. SÉLECTION DES VARIABLES ---
# On se concentre sur les colonnes utiles et on met la date en Index (requis pour les Time Series)
# Target (Y) = 'Volatility'
# Predictor (X) = 'mean' (Le score sentiment brut)
data = df_combined[['Date', 'Volatility', 'mean']].set_index('Date').sort_index()
data = data.rename(columns={'mean': 'Sentiment'}) # Renommage pour clarté
data = data.dropna() # Sécurité

print(f"Plage de test : {data.index.min().date()} au {data.index.max().date()}")
print(f"Nombre de points : {len(data)}")
print("-" * 50)

# --- 3. CHECK STATIONNARITÉ (Arbitre ADF) ---
def check_stationarity(series, name):
    result = adfuller(series)
    p_value = result[1]
    is_stationary = p_value < 0.05
    status = "✅ STATIONNAIRE" if is_stationary else "❌ NON STATIONNAIRE (Tendance détectée)"
    print(f"Test ADF pour {name.ljust(12)} : p-value = {p_value:.4f} -> {status}")
    return is_stationary

stat_vol = check_stationarity(data['Volatility'], "Volatilité")
stat_sent = check_stationarity(data['Sentiment'], "Sentiment")

# --- 4. TRANSFORMATION (Différenciation si nécessaire) ---
# Si l'un des deux n'est pas stationnaire, on différencie TOUT pour comparer des pommes avec des pommes.
if not stat_vol or not stat_sent:
    print("\n[ACTION] Données non stationnaires. Calcul des variations (Delta) J - (J-1)...")
    data_test = data.diff().dropna()
else:
    print("\n[ACTION] Données déjà stationnaires. Utilisation des niveaux bruts.")
    data_test = data

# --- 5. TEST DE GRANGER ---
# Question : Est-ce que le Sentiment aide à prédire la Volatilité ?
# Ordre des colonnes CRUCIAL dans statsmodels : [Target, Predictor] -> [Volatility, Sentiment]

MAX_LAG = 15  # On teste jusqu'à 15 jours de décalage

print("-" * 50)
print(f"LANCEMENT DU TEST DE GRANGER (Max Lag = {MAX_LAG} jours)")
print("Hypothèse : Le Sentiment 'cause' la Volatilité.\n")

# Le verbose=True affiche tous les détails statistiques. 
# On va le laisser à False et printer juste ce qui nous intéresse.
gc_res = grangercausalitytests(data_test[['Volatility', 'Sentiment']], maxlag=MAX_LAG, verbose=False)

print(f"{'LAG (Jours)':<12} | {'P-VALUE':<10} | {'CONCLUSION'}")
print("-" * 45)

found_causality = False
for lag in range(1, MAX_LAG + 1):
    # On utilise le test Chi2 (robuste)
    p_value = gc_res[lag][0]['ssr_chi2test'][1]
    
    if p_value < 0.05:
        verdict = "🟢 SIGNAL PRÉDICTIF (Significatif)"
        found_causality = True
    elif p_value < 0.10:
        verdict = "🟡 TENDANCE (Faiblement significatif)"
    else:
        verdict = "🔴 BRUIT (Pas de lien)"
        
    print(f"{lag:<12} | {p_value:.4f}     | {verdict}")

if found_causality:
    print("\n🚀 CONCLUSION : Il y a de l'Alpha ! Le sentiment précède la volatilité à certains horizons.")
else:
    print("\n📉 CONCLUSION : Pas de causalité statistique évidente sur cette période/méthode.")

Plage de test : 2007-01-03 au 2008-12-31
Nombre de points : 504
--------------------------------------------------
Test ADF pour Volatilité   : p-value = 0.3865 -> ❌ NON STATIONNAIRE (Tendance détectée)
Test ADF pour Sentiment    : p-value = 0.0000 -> ✅ STATIONNAIRE

[ACTION] Données non stationnaires. Calcul des variations (Delta) J - (J-1)...
--------------------------------------------------
LANCEMENT DU TEST DE GRANGER (Max Lag = 15 jours)
Hypothèse : Le Sentiment 'cause' la Volatilité.

LAG (Jours)  | P-VALUE    | CONCLUSION
---------------------------------------------
1            | 0.0203     | 🟢 SIGNAL PRÉDICTIF (Significatif)
2            | 0.0834     | 🟡 TENDANCE (Faiblement significatif)
3            | 0.0660     | 🟡 TENDANCE (Faiblement significatif)
4            | 0.0275     | 🟢 SIGNAL PRÉDICTIF (Significatif)
5            | 0.0388     | 🟢 SIGNAL PRÉDICTIF (Significatif)
6            | 0.0508     | 🟡 TENDANCE (Faiblement significatif)
7            | 0.0555     | 🟡 TENDANC

/Users/sojoy/Desktop/NLP-Financial-Crisis-Modeling/quant_nlp_env/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/sojoy/Desktop/NLP-Financial-Crisis-Modeling/quant_nlp_env/lib/python3.9/site-packages/numpy/linalg/_linalg.py:3220: RuntimeWarning: divide by zero encountered in matmul
  return _core_matmul(x1, x2)
/Users/sojoy/Desktop/NLP-Financial-Crisis-Modeling/quant_nlp_env/lib/python3.9/site-packages/numpy/linalg/_linalg.py:3220: RuntimeWarning: overflow encountered in matmul
  return _core_matmul(x1, x2)
/Users/sojoy/Desktop/NLP-Financial-Crisis-Modeling/quant_nlp_env/lib/python3.9/site-packages/numpy/linalg/_linalg.py:3220: RuntimeWarning: invalid value encountered in matmul
  return _core_matmul(x1, x2)
